In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix,classification_report,precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
train_identity = pd.read_csv('train_identity.csv')
train_transaction = pd.read_csv('train_transaction.csv')

In [3]:
train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [4]:
train_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
train = train_transaction.merge(train_identity,on='TransactionID',how='left')

In [6]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [7]:
objectlist = []
for key, value in train.items():
  if train[key].dtype=='float64':
     train[key].fillna(0.0,inplace=True)
  elif train[key].dtype=='int64':
     train[key].fillna(0,inplace=True)
  else:
    train[key].fillna('empty',inplace=True)
    objectlist.append(key)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Columns: 434 entries, TransactionID to DeviceInfo
dtypes: float64(399), int64(4), object(31)
memory usage: 1.9+ GB


In [9]:
train.isnull().any().sum()

0

In [10]:
count = 0
for string in objectlist:
    if count < 4:
        df = pd.get_dummies(train[string],drop_first=True)
        train=pd.concat([train,df],axis=1)
    train.drop([string],axis=1,inplace=True)
    count += 1

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Columns: 474 entries, TransactionID to ymail.com
dtypes: float64(399), int64(4), uint8(71)
memory usage: 1.8 GB


In [12]:
target_array = train['isFraud'].values
feature_array = train.drop(['TransactionID','isFraud'],axis=1).values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(feature_array,target_array, 
                                                    test_size = 0.3, 
                                                    random_state = 42, 
                                                    stratify = target_array)

In [13]:
log_reg = LogisticRegression()

In [15]:
log_reg.fit(X_train,y_train)

/Users/minhtet/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/minhtet/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
y_pred = log_reg.predict(X_test)

In [24]:
y_pred = rfc.predict(X_test)

In [25]:
print("Accuracy:", rfc.score(X_test,y_test))
print(classification_report(y_test,y_pred))
confusion_matrix(y_test,y_pred)

Accuracy: 0.9783813684650207
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    170963
           1       0.92      0.42      0.57      6199

    accuracy                           0.98    177162
   macro avg       0.95      0.71      0.78    177162
weighted avg       0.98      0.98      0.97    177162



array([[170749,    214],
       [  3616,   2583]])

In [19]:
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
 
# Predict on training set
lr_pred = lr.predict(X_test)

from sklearn.metrics import f1_score
print ("f1 score:", f1_score(y_test,y_pred))

from sklearn.metrics import recall_score
print ("recall score:", recall_score(y_test,y_pred))

/Users/minhtet/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


f1 score: 0.010766307789740344
recall score: 0.005484755605742862


In [20]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
print ("accuracy score:", accuracy_score(y_test, rfc_pred))
print ("f1 score:", f1_score(y_test, rfc_pred))
print ("recall score:", recall_score(y_test, rfc_pred))

accuracy score: 0.9783813684650207
f1 score: 0.5742552245442419
recall score: 0.4166801096951121


In [26]:
from sklearn.utils import resample
import pandas as pd

y = train.isFraud
X = train.drop('isFraud', axis=1)
X = pd.concat([pd.Series(X_train), pd.Series(y_train)], axis=1)
not_fraud = X[X.isFraud==0]
fraud = X[X.isFraud==1]

fraud_upsampled = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(not_fraud), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_fraud, fraud_upsampled])

# check new class counts
upsampled.Class.value_counts()

# trying logistic regression again with the balanced dataset
y_train = upsampled.Class
X_train = upsampled.drop('Class', axis=1)

upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

upsampled_pred = upsampled.predict(X_test)

# Checking accuracy
print (accuracy_score(y_test, upsampled_pred))

    
# f1 score
print (f1_score(y_test, upsampled_pred))
  
    
print (recall_score(y_test, upsampled_pred))
  


Exception: Data must be 1-dimensional

In [ ]:
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([not_fraud_downsampled, fraud])

# checking counts
downsampled.Class.value_counts()

# trying logistic regression again with the undersampled dataset

y_train = downsampled.Class
X_train = downsampled.drop('Class', axis=1)

undersampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

undersampled_pred = undersampled.predict(X_test)

# Checking accuracy
print (accuracy_score(y_test, undersampled_pred))

# f1 score
print (f1_score(y_test, undersampled_pred))

print (recall_score(y_test, undersampled_pred))
    

# Test Set Prediction

In [22]:
test_identity = pd.read_csv('test_identity.csv')
test_transaction = pd.read_csv('test_transaction.csv')

In [23]:
test = test_transaction.merge(test_identity,on='TransactionID',how='left')

In [27]:
objectlist = []
for key, value in test.items():
  if test[key].dtype=='float64':
     test[key].fillna(0.0,inplace=True)
  elif test[key].dtype=='int64':
     test[key].fillna(0,inplace=True)
  else:
    test[key].fillna('empty',inplace=True)
    objectlist.append(key)

In [28]:
count = 0
for string in objectlist:
    if count < 4:
        df = pd.get_dummies(test[string],drop_first=True)
        test=pd.concat([test,df],axis=1)
    test.drop([string],axis=1,inplace=True)
    count += 1

In [14]:
ids = test['TransactionID'].values
feature_array = test.drop(['TransactionID'],axis=1).values

In [15]:
y_pred = rfc.predict(feature_array)

In [16]:
proba = rfc.predict_proba(feature_array)

In [17]:
output_df = pd.DataFrame({'TransactionID':ids,'isFraud':proba[:,0]})

In [ ]:
output_df

In [18]:
output_df.to_csv('submission.csv',index = False)